In [9]:
!pip install vtk
!pip install plotly 

In [10]:
# Installing the required libraries

import plotly.subplots as ps
from plotly.subplots import make_subplots
import vtk
import numpy as np
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, fixed, FloatSlider, interactive, widgets
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [11]:
# Reading the vti file

reader = vtk.vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update() # Updating the reader to read the data

data = reader.GetOutput() # Getting the output data from the reader
# print(data)

In [12]:
# Function to convert VTK array to numpy array

def makearray(array):
    n = array.GetSize()
    a = np.zeros(n)
    for i in range(n):
        a[i] = array.GetTuple1(i)
    return a

In [13]:
# Getting scalar data from the vtk reader

scalar = data.GetPointData().GetScalars()
# print(scalar)
np_scalar_data = makearray(scalar)
# print(np_scalar_data)

In [14]:
dimensions = data.GetDimensions()
spacing = data.GetSpacing()
# Reshaping numpy_scalar_data array to match the dimensions of the data
np_scalar_data = np_scalar_data.reshape(dimensions,order='F')

In [15]:
# Creating mesh grid for isosurface
x, y, z = np.mgrid[:dimensions[0], :dimensions[1], :dimensions[2]]

In [21]:
# Creating isosurface plot

isosurface = go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=np_scalar_data.flatten(),
    isomin=0.0,
    isomax=0.0,
    showscale=False,
    colorscale='plasma',
    cmin = np_scalar_data.min(),
    cmax = np_scalar_data.max(),
)
layout = go.Layout(
    scene=dict(
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False), 
    )
)
fig2 = go.FigureWidget(data=[isosurface], layout=layout)
fig2.update_layout(width=500, height=500)


# Calculating histogram data
data_array = reader.GetOutput().GetPointData().GetArray(0)
num_points = reader.GetOutput().GetNumberOfPoints()
values = np.zeros(num_points)
for i in range(num_points):
    values[i] = data_array.GetValue(i)

histogram = np.histogram(values, bins=30)  

# Creating histogram plot
fig = go.FigureWidget(data=[go.Histogram(x=values, nbinsx=30)])
fig.update_layout(
    title='Histogram',
    xaxis_title='Vortex scalar value',
    yaxis_title='Frequency',
    width=500,
    height=500
)


# Creating Slider for isosurface
slide = widgets.FloatSlider(value=0.0, min=np.min(np_scalar_data), max=np.max(np_scalar_data), step=0.01, description="Isovalue", continuous_update=False)

# Creating reset button
def reset_button_clicked(b):
    fig2.data[0].isomin = 0.0
    fig2.data[0].isomax = 0.0
    slide.value = 0.0
    fig.data[0].x = values

def up_surface(isovalue):
    fig2.data[0].isomin = isovalue.new
    fig2.data[0].isomax = isovalue.new
    hist_new_data  = [i for i in values if i>=isovalue.new-0.25 and i<=isovalue.new+0.25 ]
    fig.data[0].x = hist_new_data

button = widgets.Button(description='Reset')
button.on_click(reset_button_clicked)
slide.observe(up_surface, names='value')

final_fig = widgets.VBox([
    widgets.HBox([slide,button]),
    widgets.HBox([fig2, fig])]
)

# Displaying the final image
display(final_fig)